In [545]:
import urllib.request, json, xmltodict
import numpy as np
import pandas as pd
import re
import gensim
import pymorphy2

In [546]:
reviews = pd.DataFrame.from_csv("./res.csv", encoding="cp1251")
reviews

,rating,content
0,1,"В описании приложения, написали что есть русск..."
1,1,Пожалуйста сделайте русский язык!!!!!
2,2,"Зашла в игру, поиграла ровно минуту и всё, эне..."
3,2,"Игра интересная, но как же достал connect erro..."
4,5,"Будьте любезны, если есть возможность добавьте..."
5,1,Where is the Russian language? why the descrip...
6,4,Без русского играть невозможно. От выборы зави...
7,5,"Почему игра не запускается, перекидывает на ст..."
8,1,Когда же ждать заявленный в описании русский я...
9,2,"Русского языка нет, но английский понимаю - та..."


In [547]:
pm2_to_univ_POS = {
    "NOUN" : "NOUN",
    "ADJF" : "ADJ",
    "ADJS" : "ADJ",
    "COMP" : "ADJ",
    "VERB" : "VERB",
    "INFN" : "VERB",
    "PRTF" : "ADJ",
    "PRTS" : "ADJ",
    "GRND" : "ADV",
    "NUMR" : "NUM",
    "ADVB" : "ADV",
    "NPRO" : "PRON",
    "PRED" : "ADV",
    "PREP" : "ADP",
    "CONJ" : "CCONJ",
    "PRCL" : "PART",
    "INTJ" : "INTJ"
} 

In [548]:
    #drop NaN reviews
#reviews.drop(reviews[pd.isnull(reviews.content)].index, inplace=True)
    #drop empty reviews
#reviews.drop(reviews[reviews.content == ""].index, inplace=True)
    #drop eng reviews
#reviews.drop(reviews[reviews.content.str.contains ("[A-Za-z]")].index, inplace=True)

reviews

,rating,content
0,1,"В описании приложения, написали что есть русск..."
1,1,Пожалуйста сделайте русский язык!!!!!
2,2,"Зашла в игру, поиграла ровно минуту и всё, эне..."
3,2,"Игра интересная, но как же достал connect erro..."
4,5,"Будьте любезны, если есть возможность добавьте..."
5,1,Where is the Russian language? why the descrip...
6,4,Без русского играть невозможно. От выборы зави...
7,5,"Почему игра не запускается, перекидывает на ст..."
8,1,Когда же ждать заявленный в описании русский я...
9,2,"Русского языка нет, но английский понимаю - та..."


In [549]:
#rand_200 = reviews.sample(200)
#rand_200.to_csv("rand200.csv")


In [550]:
morph = pymorphy2.MorphAnalyzer()

def strtonormalform(line, should_delete = {'PREP', 'CONJ', 'PRCL', 'INTJ'}):
    ans = ''
    words = re.findall(r"[\w]+[\w|-]*", line)
    for word in words:
        lexInfo = morph.parse(word)[0]
        lex = lexInfo.normal_form
        if (lexInfo.tag.POS not in should_delete) and (re.match(r"[-+]?\d+$", lex) is None) :#союзы, предлоги и т.п. не сохраняем
            ans = ans + ' ' + lex
            if (lexInfo.tag.POS):
                ans = ans + "_" + lexInfo.tag.POS
        
    return ans

def normalize_revs(revs):
    #drop NaN reviews
    revs.drop(revs[pd.isnull(revs.content)].index, inplace=True)
    #drop empty reviews
    revs.drop(revs[revs.content == ""].index, inplace=True)
    #drop eng reviews
    revs.drop(revs[revs.content.str.contains ("[A-Za-z]")].index, inplace=True)
    
    revs['content'] = revs['content'].apply(strtonormalform)



In [551]:
normalize_revs(reviews)
reviews

,rating,content
0,1,описание_NOUN приложение_NOUN написать_VERB р...
1,1,сделать_VERB русский_ADJF язык_NOUN
2,2,зайти_VERB игра_NOUN поиграть_VERB ровно_ADVB...
4,5,быть_VERB любезный_ADJS возможность_NOUN доба...
6,4,русский_ADJF играть_INFN невозможно_ADVB выбо...
7,5,почему_ADVB игра_NOUN запускаться_VERB переки...
8,1,ждать_INFN заявить_PRTF описание_NOUN русский...
9,2,русский_ADJF язык_NOUN нет_PRED английский_AD...
10,5,товарищ_NOUN учить_VERB иностранный_ADJF язык...
12,1,заявить_PRTS русский_ADJF язык_NOUN заходить_...


In [396]:
from sklearn.feature_extraction.text import CountVectorizer

data = reviews.content.values.astype('U')
cnt_vec = CountVectorizer()
transformed_data = cnt_vec.fit_transform(data)

freqs_dict = {"names" : cnt_vec.get_feature_names(), "freqs" : transformed_data.sum(axis=0).tolist()[0]}

word_freqs = pd.DataFrame(freqs_dict)
word_freqs.sort_values(by='freqs', inplace=True, ascending=False)
word_freqs

,freqs,names
10610,4793,приложение_noun
15486,4358,я_npro
1384,4226,весь_adjf
4408,2783,игра_noun
8620,2744,очень_advb
1180,2477,быть_verb
11790,1874,реклама_noun
1982,1789,вы_npro
7627,1417,обновление_noun
7298,1413,нет_pred


In [397]:
frequent_words = word_freqs[:300]
freq_nouns = frequent_words[frequent_words.names.str.endswith("advb")]
freq_nouns

,freqs,names
8620,2744,очень_advb
14219,1105,уже_advb
10181,780,почему_advb
6258,724,много_advb
1741,690,вообще_advb
3478,649,ещё_advb
14167,624,удобно_advb
13650,510,теперь_advb
10066,468,постоянно_advb
6999,448,невозможно_advb


In [398]:
def tag_words_POS(words_list):
    new_list = []
    for word in words_list:
        lexInfo = morph.parse(word)[0]
        if (lexInfo.tag.POS):
            word = word + "_" + lexInfo.tag.POS
        new_list.append(word)
    return new_list

def tag_words_POS_dict(words_dict):
    new_dict = {}
    for word in words_dict.keys():
        lexInfo = morph.parse(word)[0]
        if (lexInfo.tag.POS):
            new_word = word + "_" + lexInfo.tag.POS
        new_dict.update({new_word : words_dict[word]})
    return new_dict

In [399]:
aspect_general = ["приложение", "игра", "программа", "приложуха", "игруля"]
aspect_general = tag_words_POS(aspect_general)

aspect_stability = ["вылет", "лаг", "фриз", "загрузка", "работа", "батарея", "зарядка"]
aspect_stability = tag_words_POS(aspect_stability)

aspect_functionality = ["функция", "фича", "обновление", "функционал", "функциональность", "возможность"]
aspect_functionality = tag_words_POS(aspect_functionality)

aspect_cost = ["деньги", "стоимость", "списание", "удержание", "донат", "донатище"]
aspect_cost = tag_words_POS(aspect_cost)

aspect_usability = ["удобство", "интерфейс", "вид", "реклама", "баннер", "пункт", "меню"]
aspect_usability = tag_words_POS(aspect_usability)


print(aspect_general)
print(aspect_stability)
print(aspect_functionality)
print(aspect_cost)
print(aspect_usability)

['приложение_NOUN', 'игра_NOUN', 'программа_NOUN', 'приложуха_NOUN', 'игруля_NOUN']
['вылет_NOUN', 'лаг_NOUN', 'фриз_NOUN', 'загрузка_NOUN', 'работа_NOUN', 'батарея_NOUN', 'зарядка_NOUN']
['функция_NOUN', 'фича_NOUN', 'обновление_NOUN', 'функционал_NOUN', 'функциональность_NOUN', 'возможность_NOUN']
['деньги_NOUN', 'стоимость_NOUN', 'списание_NOUN', 'удержание_NOUN', 'донат_NOUN', 'донатище_NOUN']
['удобство_NOUN', 'интерфейс_NOUN', 'вид_NOUN', 'реклама_NOUN', 'баннер_NOUN', 'пункт_NOUN', 'меню_NOUN']


In [400]:
sent_words = {
    "ужасный" : -1,
    "интересный" : 1,
    "супер" : 1,
    "замечательный" : 1,
    "шедевр" : 1,
    "лагать" : -1,
    "вылетать" : -1,
    "пипец" : -1,
    "рекламируемый" : -1,
    "рекламный" : -1,
    "легко" : 1,
    "доступный" : 1,
    "противно" : -1 ,
    "противный" : -1,
    "хороший" : 1,
    "отличный" : 1,
    "удобный" : 1,
    "классный" : 1,
    "крутой" : 1,
    "бесплатный" : 1,
    "платный" : -1,
    "удобно" : 1,
    "невозможно" : -1,
    "ужасно" : -1, 
    "понятно" : 1,
    "быстро" : 1,
    "круто" : 1
}

sent_words = tag_words_POS_dict(sent_words)
sent_words

{'бесплатный_ADJF': 1,
 'быстро_ADVB': 1,
 'вылетать_INFN': -1,
 'доступный_ADJF': 1,
 'замечательный_ADJF': 1,
 'интересный_ADJF': 1,
 'классный_ADJF': 1,
 'круто_ADVB': 1,
 'крутой_ADJF': 1,
 'лагать_NOUN': -1,
 'легко_ADVB': 1,
 'невозможно_ADVB': -1,
 'отличный_ADJF': 1,
 'пипец_NOUN': -1,
 'платный_ADJF': -1,
 'понятно_ADVB': 1,
 'противно_PREP': -1,
 'противный_ADJF': -1,
 'рекламируемый_PRTF': -1,
 'рекламный_ADJF': -1,
 'супер_ADJF': 1,
 'удобно_ADVB': 1,
 'удобный_ADJF': 1,
 'ужасно_ADVB': -1,
 'ужасный_ADJF': -1,
 'хороший_ADJF': 1,
 'шедевр_NOUN': 1}

In [865]:
test_df = pd.DataFrame.from_csv("./200_random_revs.csv", encoding="cp1251")
test_df

,rating,content,sent_gen,sent_stab,sent_func,sent_price,sent_use
15126,5,Приложуха норм!,1,0,0,0,0
15627,5,"Круто, еле нашёл, слава богу, все работает!!!)...",1,1,0,0,0
17769,1,Смысл скачивать приложение если там без подпис...,0,0,-1,0,0
14739,4,Добавьте пожалуйста карту Тайланд. В этом есть...,0,0,-1,0,0
19606,5,"Отличное приложение, отличные водители, часто ...",1,0,0,0,0
14520,5,Нужно найти категорию (например велопрокат). П...,0,-1,0,0,0
1563,3,Купил за 119 р норм. Сейчас скидон. Она стоит ...,-1,0,0,1,0
7771,5,Все супер,1,0,0,0,0
20947,3,После обновления на новый интерфейс постоянные...,0,-1,0,0,0
17015,5,Отлично,1,0,0,0,0


In [866]:
normalize_revs(test_df)
test_df

,rating,content,sent_gen,sent_stab,sent_func,sent_price,sent_use
15126,5,приложух_NOUN норма_NOUN,1,0,0,0,0
15627,5,круто_ADVB еле_ADVB найти_VERB слава_NOUN бог...,1,1,0,0,0
17769,1,смысл_NOUN скачивать_INFN приложение_NOUN там...,0,0,-1,0,0
14739,4,добавить_VERB карта_NOUN тайланд_NOUN это_NPR...,0,0,-1,0,0
19606,5,отличный_ADJF приложение_NOUN отличный_ADJF в...,1,0,0,0,0
14520,5,нужно_PRED найти_INFN категория_NOUN велопрок...,0,-1,0,0,0
1563,3,купить_VERB р норма_NOUN сейчас_ADVB скидон_N...,-1,0,0,1,0
7771,5,весь_ADJF супер_ADJF,1,0,0,0,0
20947,3,обновление_NOUN новый_ADJF интерфейс_NOUN пос...,0,-1,0,0,0
17015,5,отлично_ADVB,1,0,0,0,0


In [867]:
test_df.content = test_df.content.str.split()

In [868]:
def find_asp_terms (revs, asp_name, asp_list, inplace=True):
    if (not inplace):
        revs = revs.copy()
    revs[asp_name+'_asp_pos'] = np.empty((len(revs), 0)).tolist()
    for index, row in revs.iterrows():
        for word in asp_list:
            if (word in row.content):
                revs.loc[index, asp_name+'_asp_pos'].append(row.content.index(word))
                
                #df1.loc[index, asp_name+'_asp_pos'] = df1.loc[index, asp_name+'_asp_pos'] + [row.content.index(word)]



find_asp_terms(test_df, asp_name="sent_gen", asp_list=aspect_general)
find_asp_terms(test_df, asp_name="sent_stab", asp_list=aspect_stability)
find_asp_terms(test_df, asp_name="sent_func", asp_list=aspect_functionality)
find_asp_terms(test_df, asp_name="sent_price", asp_list=aspect_cost)
find_asp_terms(test_df, asp_name="sent_use", asp_list=aspect_usability)

test_df

,rating,content,sent_gen,sent_stab,sent_func,sent_price,sent_use,sent_gen_asp_pos,sent_stab_asp_pos,sent_func_asp_pos,sent_price_asp_pos,sent_use_asp_pos
15126,5,"[приложух_NOUN, норма_NOUN]",1,0,0,0,0,[],[],[],[],[]
15627,5,"[круто_ADVB, еле_ADVB, найти_VERB, слава_NOUN,...",1,1,0,0,0,[],[],[],[],[]
17769,1,"[смысл_NOUN, скачивать_INFN, приложение_NOUN, ...",0,0,-1,0,0,[2],[],[],[],[]
14739,4,"[добавить_VERB, карта_NOUN, тайланд_NOUN, это_...",0,0,-1,0,0,[],[],[],[],[]
19606,5,"[отличный_ADJF, приложение_NOUN, отличный_ADJF...",1,0,0,0,0,[1],[],[],[],[]
14520,5,"[нужно_PRED, найти_INFN, категория_NOUN, велоп...",0,-1,0,0,0,[],[],[],[],[]
1563,3,"[купить_VERB, р, норма_NOUN, сейчас_ADVB, скид...",-1,0,0,1,0,[],[],[],[],[]
7771,5,"[весь_ADJF, супер_ADJF]",1,0,0,0,0,[],[],[],[],[]
20947,3,"[обновление_NOUN, новый_ADJF, интерфейс_NOUN, ...",0,-1,0,0,0,[5],[],[0],[],[2]
17015,5,[отлично_ADVB],1,0,0,0,0,[],[],[],[],[]


In [869]:
def find_sents (revs, asp_name, sent_list, win_size = 2, inplace=True):
    if (not inplace):
        revs = revs.copy()
        
    revs[asp_name+'_score'] = [0]*len(revs)
    
    for index, row in revs.iterrows():
        
        for word in sent_words:
            for pos in row.ix[asp_name+'_asp_pos']:   
                min_pos = max(0, pos-win_size)
                list_len = len(row['content']) -1
                max_pos = min(pos+win_size, list_len)
                if (word in row['content'][min_pos:max_pos]):
                    revs.loc[index, asp_name+'_score'] += sent_list[word]

find_sents(revs=test_df, asp_name="sent_gen", sent_list=sent_words)
find_sents(revs=test_df, asp_name="sent_stab", sent_list=sent_words)
find_sents(revs=test_df, asp_name="sent_func", sent_list=sent_words)
find_sents(revs=test_df, asp_name="sent_price", sent_list=sent_words)
find_sents(revs=test_df, asp_name="sent_use", sent_list=sent_words)


In [870]:
test_df

,rating,content,sent_gen,sent_stab,sent_func,sent_price,sent_use,sent_gen_asp_pos,sent_stab_asp_pos,sent_func_asp_pos,sent_price_asp_pos,sent_use_asp_pos,sent_gen_score,sent_stab_score,sent_func_score,sent_price_score,sent_use_score
15126,5,"[приложух_NOUN, норма_NOUN]",1,0,0,0,0,[],[],[],[],[],0,0,0,0,0
15627,5,"[круто_ADVB, еле_ADVB, найти_VERB, слава_NOUN,...",1,1,0,0,0,[],[],[],[],[],0,0,0,0,0
17769,1,"[смысл_NOUN, скачивать_INFN, приложение_NOUN, ...",0,0,-1,0,0,[2],[],[],[],[],0,0,0,0,0
14739,4,"[добавить_VERB, карта_NOUN, тайланд_NOUN, это_...",0,0,-1,0,0,[],[],[],[],[],0,0,0,0,0
19606,5,"[отличный_ADJF, приложение_NOUN, отличный_ADJF...",1,0,0,0,0,[1],[],[],[],[],1,0,0,0,0
14520,5,"[нужно_PRED, найти_INFN, категория_NOUN, велоп...",0,-1,0,0,0,[],[],[],[],[],0,0,0,0,0
1563,3,"[купить_VERB, р, норма_NOUN, сейчас_ADVB, скид...",-1,0,0,1,0,[],[],[],[],[],0,0,0,0,0
7771,5,"[весь_ADJF, супер_ADJF]",1,0,0,0,0,[],[],[],[],[],0,0,0,0,0
20947,3,"[обновление_NOUN, новый_ADJF, интерфейс_NOUN, ...",0,-1,0,0,0,[5],[],[0],[],[2],0,0,0,0,0
17015,5,[отлично_ADVB],1,0,0,0,0,[],[],[],[],[],0,0,0,0,0


In [871]:
import sklearn.metrics

def get_eval(y_test, y_pred):
    return {
        "accuracy": sklearn.metrics.accuracy_score(y_true = y_test, y_pred = y_pred),
        "precision" : sklearn.metrics.precision_score(y_true = y_test, y_pred = y_pred),
        "recall" : sklearn.metrics.recall_score(y_true = y_test, y_pred = y_pred),
        "f1": sklearn.metrics.f1_score(y_true = y_test, y_pred = y_pred),
    }

In [887]:
# определение наличия аспекта
def get_asp_eval(revs, asp_name):
    a1 = revs[asp_name].copy()
    a1[a1 == 0] = 0
    a1[a1 != 0] = 1
    print (asp_name, " a1\n", a1)
    
    a2 = [1 if len(i) > 0 else 0 for i in revs[asp_name + "_asp_pos"]] 
    print (asp_name, " a2\n", a2)
    return get_eval(a1, a2)

In [873]:
# определение правильности определения тональности
def get_sent_eval(revs, asp_name):
    b1 = revs[asp_name][revs[asp_name] != 0]
    #b1[b1 == 1] = 1
    #b1[b1 == -1] = 0
    #print(b1)
    b2 = []
    for i in revs[asp_name + "_score"][revs[asp_name] != 0]:
        val = 0
        if (i > 0):
            val = 1
        elif (i < 0):
            val = -1
        else:
            val = 0
        b2 += [val] 
    #b2 = [1.0 if i > 0 else 0.0 for i in revs[asp_name + "_score"][revs[asp_name] != 0]]
    #print(b2)
    return get_eval(b1, b2)

In [883]:
def get_sent_accuracy(revs, asp_name):
    b1 = revs[asp_name][revs[asp_name] != 0].copy()
    #b1[b1 == 1] = 1
    #b1[b1 == -1] = 0
    b2 = []
    for i in revs[asp_name + "_score"][revs[asp_name] != 0]:
        val = 0
        if (i > 0):
            val = 1
        elif (i < 0):
            val = -1
        else:
            val = 0
        b2 += [val] 
    #b2 = [1.0 if i > 0 else 0.0 for i in revs[asp_name + "_score"][revs[asp_name] != 0]]
    return {"accuracy" : sklearn.metrics.accuracy_score(y_true = b1.astype(int), y_pred = np.array(b2, dtype=int)),
            "precision" : np.nan,
            "recall" : np.nan,
            "f1": np.nan
           }

In [886]:
evals = pd.DataFrame(columns= ["accuracy", "precision", "recall", "f1"])
evals.loc["Определение общих аспектов"] = get_asp_eval(test_df, "sent_gen")
evals.loc["Определение аспекта стабильности"] = get_asp_eval(test_df, "sent_stab")
evals.loc["Определение аспекта функциональности"] = get_asp_eval(test_df, "sent_func")
evals.loc["Определение аспекта цены"] = get_asp_eval(test_df, "sent_price")
evals.loc["Определение аспекта удобства"] = get_asp_eval(test_df, "sent_use")
#evals.append(get_asp_eval(test_df, "sent_gen"))
evals.loc["Определение тональности общих аспектов"] = get_sent_accuracy(test_df, "sent_gen")
evals.loc["Определение тональности аспекта стабильности"] = get_sent_accuracy(test_df, "sent_stab")
evals.loc["Определение тональности аспекта функциональности"] = get_sent_accuracy(test_df, "sent_func")
evals.loc["Определение тональности аспекта цены"] = get_sent_accuracy(test_df, "sent_price")
evals.loc["Определение тональности аспекта удобства"] = get_sent_accuracy(test_df, "sent_use")
evals

,accuracy,precision,recall,f1
Определение общих аспектов,0.633333,0.606061,0.500000,0.547945
Определение аспекта стабильности,0.711111,0.500000,0.038462,0.071429
Определение аспекта функциональности,0.738889,0.450000,0.200000,0.276923
Определение аспекта цены,0.905556,0.000000,0.000000,0.000000
Определение аспекта удобства,0.905556,0.647059,0.500000,0.564103
Определение тональности общих аспектов,0.250000,NaN,NaN,NaN
Определение тональности аспекта стабильности,0.000000,NaN,NaN,NaN
Определение тональности аспекта функциональности,0.000000,NaN,NaN,NaN
Определение тональности аспекта цены,0.000000,NaN,NaN,NaN
Определение тональности аспекта удобства,0.045455,NaN,NaN,NaN
